# Download project

How to download Project (images + annotations) from Supervisely to local folder

**Input**:
- Existing Project in Supervisely

**Output**:
- Supervisely format Project in local folder

## Configuration

Edit the following settings for your own case

In [1]:
import supervisely_lib as sly
import os
from tqdm import tqdm

In [2]:
team_name = "jupyter_tutorials"
workspace_name = "cookbook"
project_name = "tutorial_project"

save_directory = './tutorial_project'

# Obtain server address and your api_token from environment variables
# Edit those values if you run this notebook on your own PC
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [4]:
# get IDs of team, workspace and project by names

team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    raise RuntimeError("Project {!r} not found".format(project_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))
print("Project: id={}, name={}".format(project.id, project.name))

Team: id=30, name=jupyter_tutorials
Workspace: id=76, name=cookbook
Project: id=898, name=tutorial_project


## Create local Project and download remote images

In [5]:
# create Project object for writing data in supervisely format  
project_fs = sly.Project(save_directory, sly.OpenMode.CREATE)

In [6]:
# download meta of remote project and save it to directory 
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)
project_fs.set_meta(meta)

In [7]:
# iterate over remote datasets and images, download image and corresponding annotation, save item (img+ann pair) to directory 
for dataset in api.dataset.get_list(project.id):
    dataset_fs = project_fs.create_dataset(dataset.name)
    
    images = api.image.get_list(dataset.id)
    with tqdm(total=len(images), desc="Process") as progress_bar:
        for batch in sly.batched(images):
            image_ids = [image_info.id for image_info in batch]
            image_names = [image_info.name for image_info in batch]
            
            #download images in numpy format
            img_nps = api.image.download_nps(dataset.id, image_ids)
            
            #download annotations in json format
            ann_infos = api.annotation.download_batch(dataset.id, image_ids)
            ann_jsons = [ann_info.annotation for ann_info in ann_infos]
            
            for name, img, ann in zip(image_names, img_nps, ann_jsons):
                dataset_fs.add_item_np(name, img, ann)

            progress_bar.update(len(batch))

Process: 100%|██████████| 2/2 [00:00<00:00, 14.60it/s]


In [8]:
print("Project {!r} has been successfully downloaded".format(project_name))
print("Total number of images: ", project_fs.total_items)

Project 'tutorial_project' has been successfully downloaded
Total number of images:  5


In [9]:
# Create a tar archive to download the project as a single tar file from jupyter lab
sly.fs.archive_directory('tutorial_project', tar_path='tutorial_project.tar')

# Remove the project folder after packing it into a tar file.
sly.fs.remove_dir('tutorial_project')